In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

MessageError: ignored

# 필요한 모듈 생성

In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 데이터 불러오기

In [14]:
train = pd.read_csv('C:/Users/hi/Desktop/YBIGTA/5_FE/train.csv')
train = train.drop('id', axis=1)

In [15]:
print(train.shape)
train.head()

(17480, 15)


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0


# Feature Engineering

## 결측치 확인

In [16]:
# 두 코드 모두 가능!
train.info()
train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17480 entries, 0 to 17479
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             17480 non-null  int64 
 1   workclass       15644 non-null  object
 2   fnlwgt          17480 non-null  int64 
 3   education       17480 non-null  object
 4   education.num   17480 non-null  int64 
 5   marital.status  17480 non-null  object
 6   occupation      15637 non-null  object
 7   relationship    17480 non-null  object
 8   race            17480 non-null  object
 9   sex             17480 non-null  object
 10  capital.gain    17480 non-null  int64 
 11  capital.loss    17480 non-null  int64 
 12  hours.per.week  17480 non-null  int64 
 13  native.country  16897 non-null  object
 14  target          17480 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 2.0+ MB


age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
target               0
dtype: int64

## 결측치 제거

In [17]:
train_1 = train.dropna()
print("결측치 제거 전 데이터 개수 : ", train.shape[0])
print("결측치 제거 후 데이터 개수 : ", train_1.shape[0])

결측치 제거 전 데이터 개수 :  17480
결측치 제거 후 데이터 개수 :  15081


In [18]:
# 결측치가 모두 사라진것을 볼 수 있음
train_1.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
target            0
dtype: int64

## 결측치 대체

In [19]:
####결측치를 0으로 대체####
train_2 = train.fillna(0)
print("결측치 대체 전 데이터 개수 : ", train.shape[0])
print("결측치 대체 후 데이터 개수 : ", train_2.shape[0])

결측치 대체 전 데이터 개수 :  17480
결측치 대체 후 데이터 개수 :  17480


- 결측치를 0으로 대체해줌으로써 데이터 개수는 기존 데이터와 같은것을 볼 수 있음

In [20]:
####결측치를 Mean or Median으로 대체####
train_3 = train.fillna(train.mean())
train_4 = train.fillna(train.median())

C:\Users\hi\AppData\Local\Temp/ipykernel_30016/844415896.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_3 = train.fillna(train.mean())
C:\Users\hi\AppData\Local\Temp/ipykernel_30016/844415896.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_4 = train.fillna(train.median())


In [21]:
####결측치를 가장 빈도수가 높은 값으로 대체####
print(train['workclass'].value_counts())
print(train['occupation'].value_counts())
print(train['native.country'].value_counts())

Private             11568
Self-emp-not-inc     1272
Local-gov            1053
State-gov             659
Self-emp-inc          594
Federal-gov           485
Never-worked            7
Without-pay             6
Name: workclass, dtype: int64
Exec-managerial      2113
Craft-repair         2101
Prof-specialty       2085
Adm-clerical         1893
Sales                1829
Other-service        1677
Machine-op-inspct    1040
Transport-moving      785
Handlers-cleaners     695
Farming-fishing       508
Tech-support          475
Protective-serv       350
Priv-house-serv        83
Armed-Forces            3
Name: occupation, dtype: int64
United-States                 15393
Mexico                          355
Philippines                     108
Germany                          75
Canada                           63
Puerto-Rico                      59
El-Salvador                      58
Cuba                             49
India                            47
England                          46
Jamaica

- 일단 결측치가 존재하는 Feature에서 어떤 unique값이 가장 빈도수가 높은지 찾아봄
    - workclass : Private
    - occupation : Exec-managerial
    - native.country : United-States

In [22]:
# 빈도수가 높은 값으로 채워줌
train_5 = train.copy()
train_5.loc[train['workclass'] != train['workclass'],'workclass'] = 'Private'
train_5.loc[train['occupation'] != train['occupation'],'occupation'] = 'Exec-managerial'
train_5.loc[train['native.country'] != train['native.country'],'native.country'] = 'United-States'
print("결측값 대체 전 workclass의 Private 값 개수 : ", train.workclass.value_counts()[0])
print("결측값 대체 후 workclass의 Private 값 개수 : ", train_5.workclass.value_counts()[0])

결측값 대체 전 workclass의 Private 값 개수 :  11568
결측값 대체 후 workclass의 Private 값 개수 :  13404


- 결측치를 빈도수가 가장 높았던 Private으로 대체해줌으로써, Private값의 개수가 늘었음을 확인할 수 있음

In [23]:
####결측치를 임의의 값으로 대체####
train_6 = train.fillna(method = 'pad')
train_7 = train.fillna(method = 'bfill')

- pad : 특정한 수치를사용하지 않고, 결측치의 바로 앞에 있는 value를 채워넣는 방식
- bfill : pad와는 반대로 각 결측치의 바로 뒤의 값으로 채워넣는 방법

# 지표변수 (Indicator variables)

In [24]:
train['age']

0        32
1        33
2        46
3        23
4        55
         ..
17475    35
17476    30
17477    71
17478    41
17479    72
Name: age, Length: 17480, dtype: int64

In [27]:
oldness = []
for a in train['age']:
    if (17 <= a <= 30):
        oldness.append('young')
    elif (31 <= a <= 50):
        oldness.append('middle')
    else:
        oldness.append('old')

train['oldness'] = oldness

In [28]:
train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target,oldness
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1,middle
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0,young
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0,old


## Feature Split

In [29]:
train['date'] = '2022-02-03'
train

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target,oldness,date
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1,middle,2022-02-03
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0,young,2022-02-03
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0,old,2022-02-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17475,35,NaN,320084,Bachelors,13,Married-civ-spouse,NaN,Wife,White,Female,0,0,55,United-States,1,middle,2022-02-03
17476,30,NaN,33811,Bachelors,13,Never-married,NaN,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,0,young,2022-02-03
17477,71,NaN,287372,Doctorate,16,Married-civ-spouse,NaN,Husband,White,Male,0,0,10,United-States,1,old,2022-02-03
17478,41,NaN,202822,HS-grad,9,Separated,NaN,Not-in-family,Black,Female,0,0,32,United-States,0,middle,2022-02-03


In [30]:
year = []
month = []
day = []

for d in train['date'].str.split("-"):
    year.append(d[0])
    month.append(d[1])
    day.append(d[2])
train['year'] = year
train['month'] = month
train['day'] = day

In [31]:
train

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target,oldness,date,year,month,day
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03,2022,02,03
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1,middle,2022-02-03,2022,02,03
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03,2022,02,03
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0,young,2022-02-03,2022,02,03
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0,old,2022-02-03,2022,02,03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17475,35,NaN,320084,Bachelors,13,Married-civ-spouse,NaN,Wife,White,Female,0,0,55,United-States,1,middle,2022-02-03,2022,02,03
17476,30,NaN,33811,Bachelors,13,Never-married,NaN,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,0,young,2022-02-03,2022,02,03
17477,71,NaN,287372,Doctorate,16,Married-civ-spouse,NaN,Husband,White,Male,0,0,10,United-States,1,old,2022-02-03,2022,02,03
17478,41,NaN,202822,HS-grad,9,Separated,NaN,Not-in-family,Black,Female,0,0,32,United-States,0,middle,2022-02-03,2022,02,03


## 스케일링 Scaling

In [32]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
mm_scaler = MinMaxScaler()
sd_scaler = StandardScaler()

In [33]:
# MinMax Scaling
mm_scaler = mm_scaler.fit_transform(train['age'].values.reshape(-1,1))
train['mm_scaled_age'] = mm_scaler

# Standard Scaling
sd_scaler = sd_scaler.fit_transform(train['age'].values.reshape(-1,1))
train['sd_scaled_age'] = sd_scaler


In [34]:
train

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,...,hours.per.week,native.country,target,oldness,date,year,month,day,mm_scaled_age,sd_scaled_age
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,...,40,United-States,0,middle,2022-02-03,2022,02,03,0.205479,-0.477370
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,...,40,United-States,1,middle,2022-02-03,2022,02,03,0.219178,-0.406343
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,...,40,United-States,0,middle,2022-02-03,2022,02,03,0.397260,0.517004
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,...,30,United-States,0,young,2022-02-03,2022,02,03,0.082192,-1.116610
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,...,40,United-States,0,old,2022-02-03,2022,02,03,0.520548,1.156244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17475,35,NaN,320084,Bachelors,13,Married-civ-spouse,NaN,Wife,White,Female,...,55,United-States,1,middle,2022-02-03,2022,02,03,0.246575,-0.264290
17476,30,NaN,33811,Bachelors,13,Never-married,NaN,Not-in-family,Asian-Pac-Islander,Female,...,99,United-States,0,young,2022-02-03,2022,02,03,0.178082,-0.619423
17477,71,NaN,287372,Doctorate,16,Married-civ-spouse,NaN,Husband,White,Male,...,10,United-States,1,old,2022-02-03,2022,02,03,0.739726,2.292671
17478,41,NaN,202822,HS-grad,9,Separated,NaN,Not-in-family,Black,Female,...,32,United-States,0,middle,2022-02-03,2022,02,03,0.328767,0.161870


## 라벨 인코딩 (Label Encoding)

In [35]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [36]:
encoded = encoder.fit_transform(train['education'])

In [37]:
train['education_class'] = encoded

In [38]:
train.education_class.value_counts()

11    5566
15    3905
9     2842
12     921
8      724
1      664
7      588
0      510
5      378
14     301
6      294
2      249
10     214
4      197
3       96
13      31
Name: education_class, dtype: int64

## 원-핫 인코딩 (One-Hot Encoding)

In [39]:
pd.get_dummies(train['education'])

,10th,11th,12th,1st-4th,5th-6th,7th-8th,9th,Assoc-acdm,Assoc-voc,Bachelors,Doctorate,HS-grad,Masters,Preschool,Prof-school,Some-college
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
17476,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
17477,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
17478,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


# 과제

## Titanic 데이터를 가지고 5 개의 FE 방법들 직접 사용해보기 (오늘 피피티에 소개되지 않은 FE 방법도 괜찮음)

In [155]:
titanic = pd.read_csv('C:/Users/hi/Desktop/YBIGTA/5_FE/titanic.csv')
titanic.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [156]:
titanic.isnull().sum()
# titanic.info()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


## 1. Imputation - 결측치 age mean 값으로 대체 + Fare 결측치 median 값으로 대체 + Cabin 결측치 'unknown'으로 대체

In [157]:
titanic_new = titanic.copy()
titanic_new.Age.fillna(titanic.Age.mean(),inplace = True)
titanic_new.Fare.fillna(titanic.Fare.mean(), inplace = True)
titanic_new.Cabin.fillna("unknown", inplace =True)
titanic_new.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,unknown,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,unknown,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,unknown,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,unknown,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,unknown,S


In [158]:
titanic_new.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

## 2. Indicator Variables - cabin 값 1/0 으로 대체

In [159]:
titanic2 = titanic.copy()
titanic2

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [160]:
titanic2.Cabin.fillna('0',inplace=True)
titanic2

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,0,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,0,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,0,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,0,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,0,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,0,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,0,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,0,S


In [161]:
def cabin_1_0(cabin):
    if cabin == '0':
        return 0
    else :
        return 1 
titanic2['Cabin'] = titanic2.Cabin.apply(cabin_1_0)

In [162]:
titanic2

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,0,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,0,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,0,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,0,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,0,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,0,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,1,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,0,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,0,S


# 3. Min Max scaling 

In [163]:
from sklearn.preprocessing import MinMaxScaler
import math

mm_scaler = MinMaxScaler()

In [164]:
def toInt(x):
  return int(x)

titanic3 = titanic.copy()
titanic3['Fare'] = titanic3['Fare'].fillna(0).apply(toInt)

mm_scaler = mm_scaler.fit_transform(titanic_scaling['Fare'].values.reshape(-1,1))
titanic3['Fare'] = mm_scaler
titanic3.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,0.015282,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,0.013663,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,0.018909,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,0.016908,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,0.023984,NaN,S


# 4. One-Hot encoding 

In [165]:
titanic4 = titanic.copy()
titanic4_embark = pd.get_dummies(titanic4['Embarked'])

In [166]:
titanic4 = pd.concat([titanic4, titanic4_embark],axis=1)
titanic4

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C,Q,S
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,0,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,1,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0,0,1
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1,0,0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0,0,1
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0,0,1


# 5. Label encoding 

In [167]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [168]:
encoded = encoder.fit_transform(titanic['Sex'])

In [169]:
titanic['sex'] = encoded

In [171]:
titanic

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,sex
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,1
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,1
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,1


In [172]:
titanic.sex.value_counts()

1    266
0    152
Name: sex, dtype: int64